<a href="https://colab.research.google.com/github/jiahfong/incoherent-thoughts/blob/develop/Realistic_Evaluation_of_Deep_Semi_Supervised_Learning_Algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Realistic Evaluation of Deep Semi-Supervised Learning Algorithms

This paper provides 6 advice on how to fairly benchmark and report methods from the SSL literature + an overview of current SOTA SSL methods.

# Current SOTA SSL methods

From (Oliver et al. 2019):

These methods are simple to implement in that they only add an additional loss term to the supervised learning's training objective without changing the model/training procedure. *The additional loss term can naturally be weighted by a hyperparameter.*

They fall in two classes: consistency regularisation and entropy minimisation. The former is derived from the **clustering assumtion** which states that nearby points are likely to have the same class (small perturbations in input does not cause a change in class). The latter is motivated by the idea of more confident predictions on unlabelled data,e.g. reducing the entropy of softmax distribution on unlabelled data, which intuitively means more generalisation as the decision boundary lies in low class probability distribution (think: SVM max margin principle).

### 1. $\Pi$-Model
To encourage consistency regularisation, this method adds a loss term that requires the distance between a network's output for different passes of an input point to be small. I.e. instead of perturbing the inputs, the model's stochasticity is responsible for the difference in output predictions.

### 2. Mean Teacher/Temporal ensembling
Similar in concept to $\Pi$-Model, but argues that the stochastic predictions can vary over time. To this end, they propose taking the exponential moving average of (model parameters, mean teacher; or output predicions, temporal ensembling) from the previous training steps to produce a more stable target prediction.

> In practice, it was found that the Mean Teacher approach outperformed Temporal Ensembling, so we will focus on it in our later experiments. (Oliver et al. 2019)

### 3. Visual Adversarial Training (VAT)
In contrast, VAT perturbs the input to get output stochasticity. This perturbation is one that would most significantly affect the output prediction, hence the name. 

### 4. Entropy minimisation (EntMin)
Adds a loss term to make the model be more confident in its predictions on unlabelled data. (Side note, Entropy *maximisation* was also proposed to regularise networks: penalise overconfident predictions).

### 5. Pseudo-labelling
Many derivatives of this method. In (Oliver et al. 2019), it was defined to be using pseudo-labels that pass a certain probability/confidence threshold on unlabelled data with the same loss as labelled examples. They cited Lee (2013), the first (?) paper on pseudo-labelling which, on the contrary, took *all* unlabelled samples and applied a scheduled-weight ($\alpha(t)$) to the loss incured using unlabelled data. Nonetheless, there are plenty other derivatives of this method. E.g. ensemble-based probability values (which prvoides more robust Pr.), MC dropout?, etc.

> Pseudo-labeling is also closely related to self-training, which differs only in the heuristics used to decide which pseudo-labels to retain. (Oliver et al. 2019) see [here](https://www.ri.cmu.edu/pub_files/pub4/rosenberg_charles_2005_1/rosenberg_charles_2005_1.pdf) and [here](https://www.tandfonline.com/doi/abs/10.1080/01621459.1975.10479874)

### 6. At-a-glance performance

In section 3 of the paper, we observe that in terms of performance on the 2-moons dataset:

$\Pi$-Model (+ Mean Teacher/Temp. Ens.) $=$ VAT $>$ Entropy Minimization $>$ Pseudo-labelling


# Adressing issues in SSL benchmarks

1. **Shared impl.** Standardising the entire suite: from hyperparameters, parameter init., data augmentation/standardisation, and training procedure. *Use the exact same underlying model when comparing SSL approaches. Differences in model structure or even implementation details can greatly impact results.* Bottomline: **choose a config. and stick to it, don't introduce unnecessary variations; these SSL methods are supposed to be agnostic to model arch., HPs, etc. anyway!**
2. **Supervised baseline.** Whilst it's intuitive that using the unlabelled pool yields higher performance, we can't take this for granted and should compare against the identical model but using only the labelled pool. *The goal of SSL should be to significantly outperform the fully-supervised setting.*
3. **Transfer learning.** Another "baseline" that mitigates the issue of having too little training data. This assumes that the model was transferred from something "identical", which does not apply all the time. *The goal of SSL should be to significantly outperform the fully-supervised setting.*
4. **Out-of-dist classes.** Common benchmarking techniques split the labelled dataset into labelled + unlabelled and pretend that the unlaballed has no labels. This implictly assumes that the unlabelled data follows the same dist. as the labelled pool, which might not always be the case, esp. in real-world datasets. This is a *realistic scenario that to our (Oliver et al. 2019) knowledge is drastically understudied.*
5. **Vary the amount of labelled _and_ unlabelled datasets.** Many methods investigate what happens as the size of the labelled varies, but less often with the unlabelled. Don't just vary one, but both.
6. **Realistically small validation sizes.** When the labelled pool is small, make sure it's not smalller than the validation set, otherwise, you might as well swap their roles!

